In [2]:
import numpy as np
import pandas as pd

In [3]:
players = pd.read_csv('../dataset light memory/players_fifa22.csv')
teams = pd.read_csv('../dataset light memory/teams_fifa22.csv')

In [8]:
list(players.columns)

['ID',
 'Name',
 'FullName',
 'Age',
 'Height',
 'Weight',
 'PhotoUrl',
 'Nationality',
 'Overall',
 'Potential',
 'Growth',
 'TotalStats',
 'BaseStats',
 'Positions',
 'BestPosition',
 'Club',
 'ValueEUR',
 'WageEUR',
 'ReleaseClause',
 'ClubPosition',
 'ContractUntil',
 'ClubNumber',
 'ClubJoined',
 'OnLoad',
 'NationalTeam',
 'NationalPosition',
 'NationalNumber',
 'PreferredFoot',
 'IntReputation',
 'WeakFoot',
 'SkillMoves',
 'AttackingWorkRate',
 'DefensiveWorkRate',
 'PaceTotal',
 'ShootingTotal',
 'PassingTotal',
 'DribblingTotal',
 'DefendingTotal',
 'PhysicalityTotal',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTac

In [17]:
['GK', 
 'LB', 'LCB', 'RCB', 'RB', 
 'CML', 'CAM', 'CMR',
 'LW', 'ST', 'RW']

['GK', 'LB', 'LCB', 'RCB', 'RB', 'LCM', 'CAM', 'RCM', 'LW', 'ST', 'RW']

In [32]:
players.columns

Index(['ID', 'Name', 'FullName', 'Age', 'Height', 'Weight', 'PhotoUrl',
       'Nationality', 'Overall', 'Potential', 'Growth', 'TotalStats',
       'BaseStats', 'Positions', 'BestPosition', 'Club', 'ValueEUR', 'WageEUR',
       'ReleaseClause', 'ClubPosition', 'ContractUntil', 'ClubNumber',
       'ClubJoined', 'OnLoad', 'NationalTeam', 'NationalPosition',
       'NationalNumber', 'PreferredFoot', 'IntReputation', 'WeakFoot',
       'SkillMoves', 'AttackingWorkRate', 'DefensiveWorkRate', 'PaceTotal',
       'ShootingTotal', 'PassingTotal', 'DribblingTotal', 'DefendingTotal',
       'PhysicalityTotal', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking',

In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class SquadBuilder(nn.module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, n_actions):
        super(SquadBuilder, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.n_actions)
        self.optimizer = optim.Adam(self.parameters(), lr = lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
    
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        return actions
        
        
        

In [89]:
class Player:
    def __init__(self, name = '', position = '', position_rating = 0, value = 0, wage = 0):
        self.name = name
        #self.overall_rating = overall_rating
        self.position = position
        self.position_rating = position_rating
        self.value = value
        #self.wage = wage
        
class Environment:
    def __init__(self, players, money):
        self.players = players[['ID', 'Name', 'ValueEUR', 'Overall', 'STRating', 'LWRating', 'LFRating', 'CFRating', 'RFRating', 'RWRating',
                                'CAMRating', 'LMRating', 'CMRating', 'RMRating', 'LWBRating',
                                'CDMRating', 'RWBRating', 'LBRating', 'CBRating', 'RBRating', 'GKRating']]
        self.money = money
        self.formation = ['GK', 
                         'LB', 'CB', 'CB', 'RB', 
                         'LM', 'CAM', 'RM',
                         'LW', 'ST', 'RW']
        self.squad = self.buildSquad()
        
    def buildSquad(self):
        squad = []
        for i in range(11):
            cur_position = self.formation.pop()
            player_id = np.random.randint(0, 1000, 1)[0]
            temp_player = Player(players.iloc[player_id]['Name'], 
                                 cur_position,
                                 players.iloc[player_id][f'{cur_position}Rating'],
                                 players.iloc[player_id]['ValueEUR'])
            squad.append(temp_player)
        return squad
    def evalSquad(self):
        sum_rating = 0
        for i in self.squad:
            sum_rating += i.position_rating
        return round(sum_rating/11, 2)

In [90]:
env = Environment(players, 1000000)

In [91]:
for player in env.squad:
    print(f'name: {player.name}, position: {player.position}, Position rating: {player.position_rating}')

name: A. Budimir, position: RW, Position rating: 69
name: M. Doherty, position: ST, Position rating: 74
name: M. Arnautović, position: LW, Position rating: 79
name: Negredo, position: RM, Position rating: 71
name: Piqué, position: CAM, Position rating: 74
name: H. Traoré, position: LM, Position rating: 77
name: M. Kranevitter, position: RB, Position rating: 75
name: S. Mandanda, position: CB, Position rating: 32
name: Domingos Duarte, position: CB, Position rating: 80
name: O. Giroud, position: LB, Position rating: 54
name: V. Rongier, position: GK, Position rating: 19


In [92]:
print(env.evalSquad())

64.0
